In [1]:
import os

In [3]:
import pathlib
import numpy as np
import keras
from keras import layers

Example with pretrained word embeddings (Keras docs)

In [ ]:

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

# import tensorflow.data as tf_data


In [ ]:
#Download the Newsgroup20 data
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)